In [5]:
%run ./pipeline.ipynb

In [6]:
#================import eeg data====================
eeg_path= './regression_dataset/eeg.txt'
eeg = pd.read_csv(eeg_path,sep='\t', skiprows=1,header=None)
#check Last column whether it is NaN
eeg = eeg.dropna(axis=1,how='all')

#================import channel data====================
channel_path = './regression_dataset/channel.txt'

channel = pd.read_csv(channel_path,sep='\t')

#================import joints data====================
joints_path = './regression_dataset/joints.txt'
joints = pd.read_csv(joints_path,sep='\t', skiprows=2,header=None)
#check the last column whether it is NaN
joints = joints.dropna(axis=1,how='all')
joints.columns = ['time','GHR', 'GKR', 'GAR', 'GHL' ,'GKL' ,'GAL' ,'PHR' ,'PKR' ,'PAR' ,'PHL' ,'PKL' ,'PAL']
#===============rename column of eeg====================
column_list = ['time']
# add channel['Phys. Chn.] into column_list drop all \t
for i in range(len(channel)-2):
    column_list.append(channel['Phys. Chn.'][i].strip())
eeg.columns = column_list
t = eeg['time']
eeg_data = eeg.drop(['time'],axis=1)

In [7]:
import numpy as np
from scipy.signal import butter, lfilter

eeg_data = np.array(eeg_data)

# Butterworth bandpass filter settings
lowcut = 0.1  # Lower frequency cutoff (Hz)
highcut = 3.0  # Upper frequency cutoff (Hz)
fs = 100.0  # Sampling frequency (Hz)
order = 4  # Filter order

# Calculate Nyquist frequency
nyquist = 0.5 * fs

# Calculate normalized cutoff frequencies
low = lowcut / nyquist
high = highcut / nyquist

# Design a Butterworth bandpass filter
b, a = butter(order, [low, high], btype='band')

# Apply the filter to each channel of the EEG data
filtered_data = lfilter(b, a, eeg_data, axis=0)
X = filtered_data[11437:15437]
y = np.array(joints['GHR'][11437:15437])
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.8)

In [ ]:
method_r = ['r_svr']
for me in method_r:
          auto(X_train, X_test, y_train, y_test,
             'BCI_0829', me,
             reduced = True, bin_reduce = True,
             file_path = './')